<a href="https://colab.research.google.com/github/contractorwolf/AngularAPIViewer/blob/master/Real_Time_Voice_Cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure GPU is enabled
Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU

I have made edits to the orginal codebase, located here:
https://github.com/CorentinJ/Real-Time-Voice-Cloning


My edits were mainly related to getting the code running fully on colab.





In [1]:
!git clone https://github.com/contractorwolf/Real-Time-Voice-Cloning.git  

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 2364 (delta 16), reused 19 (delta 8), pack-reused 2337
Receiving objects: 100% (2364/2364), 360.25 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (1300/1300), done.


In [2]:
cd Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning





install requirements for this repository


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 377.0MB 51kB/s 
     |████████████████████████████████| 1.4MB 41.6MB/s 
     |████████████████████████████████| 71kB 30.5MB/s 
     |████████████████████████████████| 245kB 56.1MB/s 
     |████████████████████████████████| 62.1MB 47.9MB/s 
     |████████████████████████████████| 204kB 63.6MB/s 
     |████████████████████████████████| 71kB 38.6MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.8-cp36-none-any.whl size=1350603 sha256=ea5408b1d50623c808ada255af8f5bb11f3194a269a4b5f4a9eb779afdf7c480
  Stored in directory: /root/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c01219b3c385a07e0
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71270 sha256=2390e2e2e44a12ae51d418dddc4b7259db6e5bc58cfbaecf06092ccf6adaa9e7
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.wh



Download the pretrained voice model that used audiobooks (text and voice) as the source. This source lacks tone, but usable. You are going to fine tune this model to your voice recording to make the cloned voice.

In [4]:
#!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc < original model hosted here
!gdown https://drive.google.com/uc?id=1dmjCLeh4w5xWOc7daA2GQPF1IqXsYTxi
  

Downloading...
From: https://drive.google.com/uc?id=1dmjCLeh4w5xWOc7daA2GQPF1IqXsYTxi
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:14, 26.4MB/s]


unpack the pretrained model

In [5]:
!unzip pretrained.zip

Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pretrained/pretrained.pt  


install required audio libraries

In [6]:
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 1s (44.3 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...


First imports libraries for recording the sample audio file (voice to be cloned).  After the files and code are loaded it immediately starts recording a file named "sample.wav" to be used in the next steps by the encoder. Read the text below until the block of code has completed (10 seconds).

READ THIS:

# The Pro is an upgraded version of the Classic. In the pro model you get a electrical system, super powerful rear hub brushless motor with tons of torque, an easy-access battery with port for phone charings, and Fat Tires with puncture-resistant casings. 



In [7]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode


print("RECORD 10 SECONDS OF TALKING, keep talking until you see 'sample.wav' output after this message")

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(filename, sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open(filename,'wb') as f:
    f.write(b)
  return 'FILE RECORDED: ' + filename

record('sample.wav', sec=10)

RECORD 10 SECONDS OF TALKING, keep talking until you see 'sample.wav' output after this message


<IPython.core.display.Javascript object>

'FILE RECORDED: sample.wav'

Executing the python file below will ask for a local file (or assume sample.wav) and create a voice signature from that sample. If you you need to upload an example for demo_colab.py you will need to upload it to the root folder and use a path like /voicesample.m4a. In the file it will ask you for the text to "say" using the voice signature.  It will then create a WAV file with a random filename (demo_output_##.wav).

NEEDED:
input for file execution should include: voice_signature_file, incoming_text, output_filename

The output will be located here: 
/content/Real-Time-Voice-Cloning/demo_output_##.wav


In [8]:
#!python demo_cli.py --no_sound
#!python demo_colab.py --no_sound
#!python demo_colab_sample.py --no_sound

!python demo_colab_sample.py --no_sound  \
  --textin="this is going to be my text for the demo it should sound like me" 


W0901 04:21:35.464736 140611097855872 deprecation_wrapper.py:119] From /content/Real-Time-Voice-Cloning/synthesizer/models/modules.py:91: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

Arguments:
    enc_model_fpath:   encoder/saved_models/pretrained.pt
    syn_model_dir:     synthesizer/saved_models/logs-pretrained
    voc_model_fpath:   vocoder/saved_models/pretrained/pretrained.pt
    low_mem:           False
    no_sound:          True
    out:               output.wav
    textin:            this is going to be my text for the demo it should sound like me
    voicein:           /content/Real-Time-Voice-Cloning/sample.wav


args.textin:  this is going to be my text for the demo it should sound like me

args.voicein:  /content/Real-Time-Voice-Cloning/sample.wav

args.out:  output.wav
Running a test of your configuration...

Found 1 GPUs available. Using GPU 0 (Tesla T4) of compute capability 7.5 with 15.8Gb total memory.

Preparin

Play output file which should be the incoming_text modeled with the voice signature file

In [0]:
import IPython.display as ipd
ipd.Audio('/content/Real-Time-Voice-Cloning/output.wav') # load a local WAV file

In [0]:
import IPython.display as ipd
ipd.Audio('sample.wav') # load a local WAV file

unused, check later


In [0]:
#!pip install youtube-dl

In [0]:
mkdir UserAudio

In [0]:
# i think this is to use a youtube video as an audio source, but it would need to have a single voice speaker
#!youtube-dl --extract-audio --audio-format wav -o "UserAudio/test.wav" link to youtube video here